# Customer Lifetime Value and Boston House Price

# 載入程式庫及必要定義

In [ ]:
try:
    from google.colab import drive, files
    in_colab = True
except ModuleNotFoundError:
    in_colab = False

if in_colab:
    home_dir = ''
    drive.mount('/content/drive')
    groot_dir = '/content/drive/My Drive/adventures/'
else:
    from pathlib import Path
    home_dir = str(Path.home())
    groot_dir = home_dir + '/Google 雲端硬碟/adventures/'

import matplotlib as mpl
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=14)
mpl.rc('ytick', labelsize=14)
mpl.rc('font', size=14)

from datetime import datetime
from dateutil.relativedelta import *
import matplotlib.pyplot as plt
import sklearn
# assert sklearn.__version__ >= "0.20"
import seaborn as sns
import pandas as pd
import numpy as np
import math
import os
import sys
import gdown
import requests
# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")
from pandas.plotting import register_matplotlib_converters

figure_dir = groot_dir + 'figure/regression/'
data_dir = groot_dir + 'regression/'

gfigure = lambda name: figure_dir + name + '.png'
output_fig = lambda name: plt.savefig( gfigure(name), dpi = 300)

local_time = lambda x, offset: x + relativedelta(hours= offset)
def local_now(hours = 8):
    return datetime.now() + relativedelta(hours = hours if in_colab else 0)

def print_now():
    return print(local_now())

def print_local_now():
    return print('Local Time:', local_now())

def DropboxLink(did, fname):
    return 'https://dl.dropboxusercontent.com/s/%s/%s' % \
    (did, fname)

def fetch_gdrive_file(fid, local_save):
    remote_url = 'https://drive.google.com/uc?id=' + fid
    gdown.download(remote_url, local_save, quiet = False)

def fetch_file_via_requests(url, save_in_dir):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter below
    output_fpath = save_in_dir + local_filename
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(output_fpath, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    # f.flush()
    return output_fpath

def acct_string(num):
    s0 = str(num)
    if len(s0) <=3:
        return s0
    num_section = int(len(s0)/3)
    remaining_start = len(s0) % 3
    s = s0[:remaining_start]
    for i in range(num_section):
        s += ',%s' % s0[remaining_start + i*3 :remaining_start + (i+1)*3]
    return s

def round_up(n, decimals=0):
    multiplier = 10 ** decimals
    return math.ceil(n * multiplier) / multiplier

def round_down(n, decimals=0):
    multiplier = 10 ** decimals
    return math.floor(n * multiplier) / multiplier

def start_plot(figsize=(10, 8), style = 'whitegrid'):
    fig = plt.figure(figsize=figsize)
    gs = fig.add_gridspec(1,1)
    plt.tight_layout()
    with sns.axes_style(style):
        ax = fig.add_subplot(gs[0,0])
    return ax

EX1DATA = '147xBeCECYur0FxDyly-oG2BqsqEH2Mxm'
EX1DATA2 = '101qw-9OkjCxwuSkBJUBaGURRWpZbFKOe'
EX5DATA = '1nNM8CN9CkRfjipRx1qJYZhSobmABVL1J'
ADVER = '1xFMcCuiMgX9VnelDtbyyV9rXBMFerx8k'
TAIWAN_CSV = '1I5yqulrZSHPSQkxT3oqt_3uVAhPolOEP'
JHU_CSSE = 'https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/'
MNIST_TRAIN = '1E-uJ0zqqAfpsVjoOSzqF5TXhDfPNlkQ5'
MNIST_TRAIN_LABEL = '13clNJ2cd2I90W3DEkDBKjZSDNNEqqx3B'
MNIST_TEST = '1zVpVHJl5YABa3qExt1K-O3WaEHXTJekg'
MNIST_TEST_LABEL = '1qci_-dqubnRN-cdrCsbYaUAxyO7_jH9z'

print('\nRunning on %s' % sys.platform)
print('Python Version', sys.version)
print('Data storage points to ==>', data_dir)

print('\nThis module is amied to leran regression basics...')
print('\nLibraries and dependenciess imported')
print_now()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted'])

# 下載資料檔案

In [ ]:
fetch_file_via_requests(
    DropboxLink('e7lsf1k6258w7co', 'ec_201012_test_4.csv'), data_dir )

fetch_file_via_requests(
    DropboxLink('fwokyefy0looizp', 'ec_201012_train_4.csv'), data_dir )

In [ ]:
import sklearn.metrics
for x in sklearn.metrics.SCORERS.keys():
    print(x)

# Preparing Data

訓練資料與測試（test）資料的名稱 ec_201012_train_4.csv 和 ec_201012_test_4.csv, 這兩個資料集的欄位相同，包括：

```
'CustomerID', 'date_size', 'date_recency', 'date_time_between',
'date_T', 'baseket_value_sum', 'baseket_value_mean',
'baseket_value_std', 'baseket_value_amax', 'baseket_value_amin',
'baseket_value_median', 'basket_size_sum', 'basket_size_mean',
'basket_size_std', 'basket_size_amax', 'basket_size_amin',
'basket_size_median', 'lag_12', 'lag_11', 'lag_10', 'lag_9', 'lag_8',
'lag_7', 'lag_6', 'lag_5', 'lag_4', 'lag_3', 'lag_2', 'lag_1', 'value'
```

最後一個欄位 <font color='brown'>‘value’</font> 是本案例需要預測的對象，是指未來一定期間內客戶購買的金額，也就是所謂的「貢獻值」。

### 資料命名規則

- CLV Case
    - train: 訓練集載入 DataFrame
    - test: 測試集載入 DataFrame
    - X_train, y_train (clv case 訓練集)
    - X_test, y_test (clv case 測試集)
- load_boston()
    - X_bos, y_bos (load_boston() 資料)
    - bos (資料打包為 Pandas DataFrame)

In [ ]:
train_csv = os.path.join(data_dir, 'ec_201012_train_4.csv')
test_csv = os.path.join(data_dir, 'ec_201012_test_4.csv')

In [ ]:
train = pd.read_csv(train_csv)
test = pd.read_csv(test_csv)

X_train = train.drop(['CustomerID','value'], axis = 1)
y_train = train.value
X_test = test.drop(['CustomerID','value'], axis = 1)
y_test = test.value

In [ ]:
from sklearn.datasets import load_boston

data = load_boston()
bos = pd.DataFrame(data = data['data'],
    columns = data['feature_names'])
bos['y'] = data['target']
X_bos = bos.drop(['y'], axis = 1)
y_bos = bos.y

- CRIM per capita crime rate by town

- ZN proportion of residential land zoned for lots over 25,000 sq.ft.

- INDUS proportion of non-retail business acres per town

- CHAS Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)

- NOX nitric oxides concentration (parts per 10 million)

- RM average number of rooms per dwelling

- AGE proportion of owner-occupied units built prior to 1940

- DIS weighted distances to five Boston employment centres

- RAD index of accessibility to radial highways

- TAX full-value property-tax rate per $10,000

- PTRATIO pupil-teacher ratio by town

- B 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town

- LSTAT % lower status of the population

- MEDV Median value of owner-occupied homes in $1000’s

## 常常需要載入的 Classes

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

def simple_ols(xvec, yvec):
    Xadd = sm.add_constant(xvec)
    model = sm.OLS(yvec, Xadd).fit()
    return model

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Modeling

## Customter Lifetime Value

In [ ]:
#檢查
# train.isnull().sum()
# test.isnull().sum()

corr = train.corr()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 14))
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, annot=True)
#CORR 太繁雜

In [ ]:
corr['value'].sort_values(ascending=False)
#屏除哪項特徵

In [ ]:
#刪掉可能對資料無意的特徵
#刪掉離群值(outliers)

train.head()

In [ ]:
from sklearn.pipeline import Pipeline, make_pipeline

the_model = make_pipeline(
    StandardScaler(),
    LinearRegression()
)

train_x = X_train
test_x = X_test
train_y = y_train
test_y = y_test

the_model.fit(train_x, train_y)
y_pred = the_model.predict(test_x)

rmse = np.sqrt(mean_squared_error(test_y, y_pred))
r_2 = r2_score(test_y, y_pred)
r_2, rmse

## load_boston

In [ ]:
corr = bos.corr()

fig, ax = plt.subplots(figsize=(16, 14))
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, annot=True)
#負相關也有用，0相關才沒用

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(X_bos, y_bos, test_size=0.2)

model = make_pipeline(
    MinMaxScaler(),
    LinearRegression()
)

model.fit(train_x, train_y)
y_pred = model.predict(test_x)

rmse = np.sqrt(mean_squared_error(test_y, y_pred))
r2 = r2_score(test_y, y_pred)
r2, rmse

In [ ]:
from statsmodels.tools.eval_measures import rmse

model = simple_ols(X_bos, y_bos)

rmse_score = rmse(y_bos, model.fittedvalues)

print(r2, rmse_score)

# Charting

In [ ]:
def start_plot(figsize=(10, 8), style = 'whitegrid'):
    fig = plt.figure(figsize=figsize)
    gs = fig.add_gridspec(1,1)
    plt.tight_layout()
    with sns.axes_style(style):
        ax = fig.add_subplot(gs[0,0])
    return ax

## 誤差值的統計分佈模型 Distribution of Residuals

- Points are **independent** of each other (residuals are uncorrelated)
- <font color='brown'>**residual ε are normally distributed with  μ = 0**</font>

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_bos, y_bos)
y_pred = model.predict(X_bos)

ax = start_plot(figsize=(8, 5))
sns.distplot(y_bos - y_pred, bins=20,
             color='darkorange', ax=ax)
ax.axvline(x=0, ls='--', c='brown')

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

ax = start_plot(figsize=(8, 5))
sns.distplot(y_test - y_pred, bins=20,
             color='darkorange', ax=ax)
ax.axvline(x=0, ls='--', c='brown')

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_true = y_test

residuals = (y_true - y_pred).sort_values(ascending=True)
ax = start_plot(style='darkgrid')
ax.plot(range(len(residuals)), residuals)
ax.axhline(y=0, ls='--', c='red')

## <font color='orange'>**預測值的範圍與分佈**<font>

###折線圖

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_true = y_test
print(type(y_true), y_true.shape)
ax = start_plot(style='darkgrid')
ax.plot(y_true, label='Observed')
ax.plot(y_pred, label='Predicted')
ax.legend(frameon=True, shadow=True)

###區間範圍圖

In [ ]:
#針對seaborn，data structure 為 DataFrame最為方便
ndf = pd.DataFrame()
ndf['Observed'] = y_true
ndf['Predicted'] = y_pred

ax = start_plot(figsize=(8, 5), style='darkgrid')
sns.boxplot(data=ndf, orient='h', ax=ax)

###誤差分布圖

In [ ]:
ax = start_plot(figsize=(8, 6))
sns.distplot(ndf.Observed)
sns.distplot(ndf.Predicted)

## $ y  - \hat y$ (預測效果) 的分析 （Regression Prediction Error）

- [YellowBrick PredictionErrorPlot](https://www.scikit-yb.org/en/latest/api/regressor/peplot.html)

![texto alternativo](https://www.scikit-yb.org/en/latest/api/regressor/peplot-1.png)

In [ ]:
!pip3 install --upgrade yellowbrick

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_true = y_test

In [ ]:
ax = start_plot(figsize=(8, 8))
sns.regplot(y_true, y_pred)
ax.scatter(y_true, y_pred, edgecolors='navy', color='orange',
     label=r'$r^2=%.4f$' % r2_score(y_true, y_pred))
lim1 = min(min(y_true), min(y_pred)) - 50
lim2 = max(max(y_true), max(y_pred)) + 50
lim = [lim1, lim2]
ax.set_xlim(lim)
ax.set_ylim(lim)
ax.plot(lim, lim, c='brown', ls='--', label=r'$y=\hat y, identity$')
ax.legend()

## Residuals Analysis 錯誤值的分析

- [Residuals Plot](https://www.scikit-yb.org/en/latest/api/regressor/residuals.html)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)
y_true_train = y_train
y_true_test = y_test
residuals_train = y_train - y_pred_train
residuals_test = y_test - y_pred_test

In [ ]:
ax = start_plot(figsize=(12, 9))
r2_train = r2_score(y_true_train, y_pred_train)
r2_test = r2_score(y_true_test, y_pred_test)
ax.scatter(y_true_train, residuals_train, label='Training Residuals ' + r'$R^2=%.4f$' % r2_train)
ax.scatter(y_true_test, residuals_test, label='Testing Residuals ' + r'$R^2=%.4f$' % r2_test)
ax.axhline(y=0, ls='--', c='r')
ax.legend(frameon=True, fancybox=True)

### My Residuals Plot

In [ ]:
import itertools, operator
from matplotlib.cm import ScalarMappable
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import gridspec

def ResidualsPlot(model, X_train, y_train, X_test=None, y_test=None, layout='h',
                  ax=None, figsize=(16,12), style='darkgrid'):

    if X_test is None:
        X_tr, X_te, y_tr, y_te = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
    else:
        X_tr, X_te, y_tr, y_te = X_train, X_test, y_train, y_test

    model.fit(X_tr, y_tr)
    yhat_train = model.predict(X_tr)
    r2_train = r2_score(y_tr, yhat_train)
    yhat_test = model.predict(X_te)
    r2_test = r2_score(y_te, yhat_test)

    fig = plt.figure(figsize=figsize)
    gs = gridspec.GridSpec(1, 2, width_ratios=[5, 1])
    plt.tight_layout()
    with sns.axes_style('whitegrid'):
        ax0 = fig.add_subplot(gs[0, 0])

    ax0.scatter(yhat_train, y_tr - yhat_train, alpha=0.7, color='navy',
        label=('training ' if X_test is not None else 'dataset examined') +
            r'$R^2=%5.3f$' % r2_train)

    ax0.scatter(yhat_test, y_te - yhat_test, alpha=0.7, color='orange', edgecolor='navy',
        label='test ' + r'$R^2=%5.3f$' % r2_test)

    ax0.set_xlabel('Predicted', fontsize=14)
    ax0.set_ylabel('Residuals', fontsize=14)
    ax0.axhline(y=0, c='r', ls='--', lw=2)
    ax0.legend(loc='best', fontsize=14, fancybox=True, shadow=True, edgecolor='navy')

    with sns.axes_style('darkgrid'):
        ax1 = fig.add_subplot(gs[0, 1])
        sns.distplot(y_train - yhat_train, color='navy', bins=30, vertical=True,
            label='training' if X_test is not None else 'examined',
            hist_kws={'edgecolor':'black'}, kde_kws={'linewidth':3})

    if X_test is not None:
        sns.distplot(y_test - yhat_test, color='darkorange', bins=30,
            label='test', vertical=True, ax=ax1, hist_kws={'edgecolor':'black'}, kde_kws={'linewidth': 3})

    ax1.legend(loc='best', fontsize=12, edgecolor='green', shadow=True, fancybox=True)
    ax1.set_xlabel('Distribution', fontsize=14)
    ax1.set_ylabel('')
    ax1.tick_params(axis='y', direction='in', labelleft=False, right=True, left=False)



In [ ]:
ResidualsPlot(LinearRegression(), X_train, y_train, X_test, y_test, figsize=(14,10))

## Residuals Analysis 錯誤值的分析 (二)

In [ ]:
ax = start_plot(figsize=(12, 9), style='darkgrid')

ax.scatter(y_pred_train, residuals_train, color='teal', label='Training Residuals ' +
           r'$R^2=%.4f$' % r2_train)
ax.scatter(y_pred_test, residuals_test, color='orange', edgecolors='navy', label='Test Residuals ' +
           r'$R^2=%.4f$' % r2_test)
ax.axhline(y=0, c='r', ls='--')
ax.legend(frameon=True, fancybox=True, fontsize=14)

## 學習曲線

In [ ]:
from sklearn.model_selection import learning_curve

#
# Simplified version of plot_learning_curves presented in
#   https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html
#

def my_plot_leaning_curves(model, X, y, cv=None, scoring = None,
        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 10)):


    fig,ax = plt.subplots(figsize = (10, 8))
    ax.set_xlabel("Training examples")
    ax.set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(model, X, y, cv=cv, n_jobs=n_jobs,
            scoring = scoring,
            train_sizes=train_sizes,
            return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve


    ax.plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    ax.plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    ax.grid(b = 'on', ls = '--', alpha = 0.8)
    ax.fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    ax.fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")

    ax.legend(loc="best", fontsize = 14, shadow =True, frameon = True)

In [ ]:
my_plot_leaning_curves(LinearRegression(), X_bos, y_bos,
    train_sizes=np.linspace(.1, 1.0, 5),
    scoring='neg_root_mean_squared_error')

# Metrics & Outliers

## Cook's Distance

- [Wikipedia](https://en.wikipedia.org/wiki/Cook%27s_distance)
- [Mathworks](https://www.mathworks.com/help/stats/cooks-distance.html#:~:text=Cook's%20distance%20is%20the%20scaled,on%20the%20fitted%20response%20values.)
- [Stackoverflow QA](https://stackoverflow.com/questions/51390196/how-to-calculate-cooks-distance-dffits-using-python-statsmodel)

- [MAPE formula](https://stats.stackexchange.com/questions/58391/mean-absolute-percentage-error-mape-in-scikit-learn/294069#294069)

- [Issue 15007](https://github.com/scikit-learn/scikit-learn/pull/15007)

>Simply said, Cook’s D is calculated by removing the ith data point from the model and recalculating the regression. All the values in the regression model are then observed whether changes have been detected after the removal of the point. This is an iterative way of examining the influence of that observation.



以 loas_boston 為例：

```
model = simple_ols(X_bos, y_bos)
infl = model.get_influence()
rdf = infl.summary_frame()
cooks_d = rdf['cooks_d']
```

In [ ]:
model = simple_ols(X_bos, y_bos)
infl = model.get_influence()
rdf = infl.summary_frame()
cooks_d = rdf['cooks_d']
rdf
ax = start_plot(figsize=(8, 5))
ax.vlines(range(len(rdf)), cooks_d, 0, color='navy')

In [ ]:
cooks_d = rdf['cooks_d'].sort_values(ascending=False)
type(cooks_d), cooks_d

- cooks distance > 3 mean(cooks distance) 可以當作outliers

- 前5% cookes distance 也可當作outliers

In [ ]:
top8 = cooks_d.index[:int(len(cooks_d.index) * 0.08)]

In [ ]:
top8 = list(top8)

In [ ]:
mu_3 = 3 * cooks_d.mean()
above_3 = cooks_d.loc[cooks_d > mu_3].index

In [ ]:
bos2 = bos.drop(above_3, axis=0)

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(LinearRegression(), X_bos, y_bos)
scores2 = cross_val_score(LinearRegression(), bos2.drop(['y'], axis=1), bos2.y)

print(scores)
print(scores.mean(), scores.std())
print('')
print(scores2)
print(scores2.mean(), scores2.std())

- [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)
- [What Is R Squared And Negative R Squared](http://www.fairlynerdy.com/what-is-r-squared/)
- [Can the multiple linear correlation coefficient be negative?](https://stats.stackexchange.com/questions/6181/can-the-multiple-linear-correlation-coefficient-be-negative)

### yellowbrick 版本

```
from yellowbrick.regressor import CooksDistance
from sklearn.datasets import load_boston

data = load_boston()
X, y = data['data'], data['target']

# Instantiate and fit the visualizer
visualizer = CooksDistance()
visualizer.fit(X, y)
visualizer.show()
```

## VIF

>Variance Inflation Factor (VIF) is used to detect the presence of multicollinearity. Variance inflation factors (VIF) measure how much the variance of the estimated regression coefficients are inflated as compared to when the predictor variables are not linearly related.

- [select_dtypes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.select_dtypes.html)
- [endog, exog, what’s that?](https://www.statsmodels.org/stable/endog_exog.html)

In [ ]:
import pandas as pd
import numpy as np
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.datasets import load_boston

def vif_scores(df):
    df.dropna()
    # df = df._get_numeric_data()
    df = df.select_dtypes(include=[np.number])
    vif = pd.DataFrame()

    vif["VIF Factor"] = \
        [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]

    vif["features"] = df.columns
    return vif

def my_mape(estimator, X, y):
    estimator.fit(X, y)
    y_pred = estimator.predict(X)
    y_true = y
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


In [ ]:
bos2 = bos.copy()
vif = vif_scores(bos2.drop(['y'], axis=1))
vif

In [ ]:
vindex = list(vif['VIF Factor'].sort_values(ascending=False).index)
vindex

In [ ]:
vif.iloc[vindex[:3]]['features']

In [ ]:
bos2 = bos2.drop(vif.iloc[vindex[:3]]['features'], axis=1)
bos2.info()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

scores = cross_val_score(LinearRegression(), X_bos, y_bos)
scores2 = cross_val_score(LinearRegression(), bos.drop(['y'], axis=1), bos2.y)

print(scores)
print(scores.mean(), scores.std())
print('')
print(scores2)
print(scores2.mean(), scores2.std())

# 降維

## Principal Component Analysis


- [線代啟示錄：主成分分析](https://ccjou.wordpress.com/2013/04/15/主成分分析/)
- [機器/統計學習:主成分分析(Principal Component Analysis, PCA)](https://medium.com/@chih.sheng.huang821/機器-統計學習-主成分分析-principle-component-analysis-pca-58229cd26e71)
- [主成分分析的原理](http://web.ntpu.edu.tw/~ccw/statmath/M_pca.pdf)
- [如何通俗易懂地讲解什么是 PCA 主成分分析？](https://www.zhihu.com/question/41120789)
- [scikit-learn PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)
- [scikit-learn User Guide: 2.5. Decomposing signals in components (matrix factorization problems)](https://scikit-learn.org/stable/modules/decomposition.html#pca)

- [Principal Component Analysis (PCA) in Python](https://www.datacamp.com/community/tutorials/principal-component-analysis-in-python)
- [Feature Selection Techniques in Machine Learning with Python](https://towardsdatascience.com/feature-selection-techniques-in-machine-learning-with-python-f24e7da3f36e)


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

# model = LinearRegression()

model = make_pipeline(
    StandardScaler(),
    LinearRegression()
)

model.fit(X_train, y_train)

yhat_train = model.predict(X_train)
yhat_test = model.predict(X_test)

TRAINING_RMSE = -np.sqrt(mean_squared_error(y_train, yhat_train))
TEST_RMSE = -np.sqrt(mean_squared_error(y_test, yhat_test))

TRAINING_SCORE = model.score(X_train, y_train)
TEST_SCORE = model.score(X_test, y_test)

print('training: score = %.4f, neg rmse = %.4f' %
      (TRAINING_SCORE, TRAINING_RMSE))
# yhat = lin.predict(X_test)
print('test: score = %.4f, neg rmse = %.4f' %
      (TEST_SCORE, TEST_RMSE))


In [ ]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
import sklearn.base.decomposition

In [ ]:
# original number of components
N = len(X_train.columns)

scores_train = np.zeros(N)
scores_test = np.zeros(N)

for i in np.arange(N, 2, -1):
    model = make_pipeline(
        StandardScaler(),
        PCA(n_components=i),
        LinearRegression()
    )
    model.fit(X_train, y_train)
    scores_train[i-1] = model.score(X_train, y_train)
    scores_test[i-1] = model.score(X_test, y_test)


In [ ]:
xdomain = np.arange(2, N, 1)
ax = start_plot(figsize=(8,8))
ax.plot(xdomain, scores_train[2:], color='orange', label='Training Scores')
ax.plot(xdomain, scores_test[2:], color='navy', label='Testing Scores')
ax.legend(loc='lower center', frameon=True, shadow=True)

# 24 components have enough performance

### Git it a try

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline, make_pipeline

def PCA_analysis(estimater, n_features, X_train, y_train,
    X_test = None, y_test = None):

    N = n_features

    scores_train = np.zeros(N)
    scores_test = np.zeros(N)

    for i in np.arange(N, 2, -1):
        model = make_pipeline(
            PCA(n_components = i),
            estimater
        )
        model.fit(X_train, y_train)
        scores_train[i-1] = model.score(X_train, y_train)
        if X_test is not None:
            scores_test[i-1] = model.score(X_test, y_test)

    xdomain = np.arange(2, N, 1)
    ax = start_plot(figsize=(10,7))
    ax.plot(xdomain, scores_train[2:], label = 'Training Scores')
    if X_test is not None:
        ax.plot(xdomain, scores_test[2:], color = 'brown',
            label = 'Test Scores')
    ax.legend(loc='lower center', frameon=True,shadow=True,fancybox=True,fontsize=14)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

model = make_pipeline(
    MinMaxScaler(),
    LinearRegression()
)
PCA_analysis(model, X_train.shape[1], X_train, y_train, X_test, y_test)